# Training Paraphrasing with BART

quellen
https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887
https://github.com/hetpandya/paraphrase-datasets-pretrained-models/blob/main/examples/t5_paraphrase_model_training_example.ipynb

## Install libraries and download the dataset

In [2]:

import evaluate
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM, PegasusForConditionalGeneration, PegasusTokenizer, AutoTokenizer
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import math
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm
import random
from transformers import pipeline
import torch

In [3]:

model_name='facebook/bart-large-cnn'
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline("text2text-generation", model=model_name)

## Load the dataset

In [11]:
df_train=pd.read_csv('data/ergbnisse/train_news.csv')
df_train

,Unnamed: 0,Zusammenfassung,Min_Kompressionsrate,Max_Kompressionsrate,Endgueltige_Kompressionsrate,länge Zusammenfassung,länge Ausgangstext,batch_texts,batch_output,text,text_rank_text,tokens_gesamt,token_text_rank,desired_compression_rate,text_rank_compression_rate,current_compression_rate,compression_difference,reduction_multiplier,ent_com_rate
0,0,The Administration of Union Territory Daman an...,34,84,100.000000,33,33,['The Administration of Union Territory Daman ...,['The Administration of Union Territory Daman ...,The Administration of Union Territory Daman an...,The Administration of Union Territory Daman an...,60,33,0.54,0.550000,0.550000,-0.010000,0.981818,0.550000
1,1,Malaika Arora slammed an Instagram user who tr...,40,90,100.000000,39,39,['Malaika Arora slammed an Instagram user who ...,['Malaika Arora slammed an Instagram user who ...,Malaika Arora slammed an Instagram user who tr...,Malaika Arora slammed an Instagram user who tr...,60,39,0.46,0.650000,0.650000,-0.190000,0.707692,0.650000
2,2,Bihar Health Minister defined virgin as being ...,36,86,97.142857,34,35,"['Earlier, Bihar Health Minister defined virgi...",['Bihar Health Minister defined virgin as bein...,The Indira Gandhi Institute of Medical Science...,"Earlier, Bihar Health Minister defined virgin ...",60,35,0.55,0.583333,0.583333,-0.033333,0.942857,0.566667
3,3,Lashkar e Taibas Kashmir commander Abu Dujana ...,35,85,76.470588,26,34,['Lashkar e Taibas Kashmir commander Abu Dujan...,['Lashkar e Taibas Kashmir commander Abu Dujan...,"Lashkar e Taibas Kashmir commander Abu Dujana,...","Lashkar e Taibas Kashmir commander Abu Dujana,...",62,34,0.32,0.566667,0.548387,-0.228387,0.583529,0.419355
4,4,Hotels in Maharashtra will train their staff t...,53,103,101.960784,52,51,['Hotels in Maharashtra will train their staff...,['Hotels in Maharashtra will train their staff...,Hotels in Maharashtra will train their staff t...,Hotels in Maharashtra will train their staff t...,60,51,0.69,0.850000,0.850000,-0.160000,0.811765,0.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,PM Narendra Modi said everyone needs to rise a...,49,99,80.851064,38,47,['PM Narendra Modi on Wednesday said everyone ...,['PM Narendra Modi said everyone needs to rise...,PM Narendra Modi on Wednesday said everyone ne...,PM Narendra Modi on Wednesday said everyone ne...,59,47,0.70,0.796610,0.796610,-0.096610,0.878723,0.644068
496,496,"A US man, who claimed to be an ex employee of ...",38,88,89.189189,33,37,"['A US man, who claimed to be an ex employee o...","['A US man, who claimed to be an ex employee o...","A US man, who claimed to be an ex employee of ...","A US man, who claimed to be an ex employee of ...",60,37,0.46,0.637931,0.616667,-0.156667,0.745946,0.550000
497,497,The film will feature Nawazuddin in the role o...,22,72,100.000000,21,21,"['Written and directed by Maitrey Bajpai, the ...",['The film will feature Nawazuddin in the role...,Nawazuddin Siddiquis first look from the upcom...,"Written and directed by Maitrey Bajpai, the fi...",54,21,0.26,0.388889,0.388889,-0.128889,0.668571,0.388889
498,498,Attorney General KK Venugopal on behalf of the...,65,115,49.206349,31,63,['Attorney General KK Venugopal on behalf of t...,['Attorney General KK Venugopal on behalf of t...,Attorney General KK Venugopal on behalf of the...,Attorney General KK Venugopal on behalf of the...,63,63,0.48,1.033333,1.000000,-0.520000,0.480000,0.492063


In [8]:
import ast
df_explotde=df_train[['batch_texts','batch_output']]
df_explotde=df_explotde.applymap(ast.literal_eval)

In [9]:
df_explotde['C'] = df_explotde.apply(lambda row: list(zip(row['batch_texts'], row['batch_output'])), axis=1)
df_explotde = df_explotde.explode('C')

# Get 'A' and 'B' values back
df_explotde[['batch_texts', 'batch_output']] = pd.DataFrame(df_explotde['C'].tolist(), index=df_explotde.index)
df = df_explotde.drop(columns=['C'])

In [10]:
df

,batch_texts,batch_output
0,The Administration of Union Territory Daman an...,The Administration of Union Territory Daman an...
1,Malaika Arora slammed an Instagram user who tr...,Malaika Arora slammed an Instagram user who tr...
2,"Earlier, Bihar Health Minister defined virgin ...",Bihar Health Minister defined virgin as being ...
3,"Lashkar e Taibas Kashmir commander Abu Dujana,...",Lashkar e Taibas Kashmir commander Abu Dujana ...
4,Hotels in Maharashtra will train their staff t...,Hotels in Maharashtra will train their staff t...
...,...,...
495,PM Narendra Modi on Wednesday said everyone ne...,PM Narendra Modi said everyone needs to rise a...
496,"A US man, who claimed to be an ex employee of ...","A US man, who claimed to be an ex employee of ..."
497,"Written and directed by Maitrey Bajpai, the fi...",The film will feature Nawazuddin in the role o...
498,Attorney General KK Venugopal on behalf of the...,Attorney General KK Venugopal on behalf of the...


# Create Test und Train Data 

In [ ]:
def token_count(text):
    tokens = text.split()
    return len(tokens)

In [ ]:
def token_count(text):
    tokens = text.split()
    return len(tokens)


def adjust_length(text):
    length = token_count(text)
    if length <20:
        min_length = length + int(length * 0.05)
        max_length = min_length +min_length
    elif length < 80:
        min_length = length + int(length * 0.05)
        max_length = min_length + 50
    elif length < 100:
        min_length = length + int(length * 0.3)
        max_length = min_length + 100
    else:
        min_length = math.ceil(length / 50) * 70
        max_length = min_length + 100
    return min_length, max_lengths

In [ ]:
def batch_sent(sentenc,splitt=180,split='. '):
    sentences = sentenc.split(split)
    # Erstellen Sie Batches von Sätzen, die weniger als 1024 Tokens enthalten
    batches = []
    batch = []
    batch_len = 0
    for sentence in sentences:
        
        sentence_len = len(tokenizer.tokenize(sentence))
        if sentence_len + batch_len > splitt:
            if sentence_len < splitt:  # überspringen Sie Sätze, die länger als 256 Tokens sind
                batches.append(batch)
                batch = [sentence]
                batch_len = sentence_len
            # wenn ein Satz alleine 1024 Tokens überschreitet, wird er übersprungen
        else:
            batch.append(sentence)
            batch_len += sentence_len
    batches.append(batch)
    return batches

In [ ]:
def paraphrase_of_text(df_s,text_name='text',komp_name='reduction_multiplier',split='. '):
    counter = 0
    df_summary_testing = pd.DataFrame(columns=['text', 'Zusammenfassung','Min_Kompressionsrate', 'Max_Kompressionsrate', 'Endgueltige_Kompressionsrate','länge Zusammenfassung','länge Ausgangstext', 'batch_texts',])
    # Teilen Sie den Text in Sätze --> für tatsächliche umsetzubng
    soplitting=True
    for _, row in tqdm(df_s.iterrows(), total=df_s.shape[0]):
     
        text = row[text_name]
        komp = row[komp_name]
        text_gesamt_list=[]

        for batch in tqdm(batch_sent(text,split=split), desc='Verarbeite Batches'):
            # Zusammenfügen der Sätze in einem Batch
            batch_text = '. '.join(batch)
            min_length_test, max_length_test = adjust_length(batch_text)
            ext_summary=summarizer(batch_text, max_length=int(round(max_length_test*komp,0)), min_length=int(round(min_length_test*komp,0)),length_penalty=100,num_beams=2)
            # Erstellen Sie einen DataFrame für die aktuellen Ergebnisse
           
            text_gesamt_list.append(ext_summary[0]['generated_text'])
        text_gesamt = '. '.join(text_gesamt_list)
        actual_compression_rate = len(text_gesamt.split(' '))/len(text.split(' '))*100
      
        df_current = pd.DataFrame({
            'text':[text],
            'Zusammenfassung': [text_gesamt],
            'Min_Kompressionsrate': [min_length_test],
            'Max_Kompressionsrate': [max_length_test],
            'Endgueltige_Kompressionsrate': [actual_compression_rate],
            'länge Zusammenfassung': [len(text_gesamt.split(' '))],
            'länge Ausgangstext': [len(text.split(' '))]
        
        })
       
        # Fügen Sie die Daten zum DataFrame hinzu
        df_summary_testing = pd.concat([df_summary_testing, df_current], ignore_index=True)
        counter += 1
        soplitting=False
    return df_summary_testing


In [ ]:
def calculate_compression(df, total_tokens_col, current_tokens_col, desired_compression_rate):
    df['current_compression_rate'] = df[current_tokens_col] / df[total_tokens_col]
    df['compression_difference'] = desired_compression_rate - df['current_compression_rate']
    df['reduction_multiplier'] = desired_compression_rate / df['current_compression_rate']
    return df


In [ ]:
df_test=calculate_compression(df_test, 'total_tokens', 'token_count', desired_compression_rate)

In [ ]:
df_summary=paraphrase_of_text(df_test[['text','reduction_multiplier']])

## Dataset train/validation/test split

In [ ]:
# datasets_train_test = 
# datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=3000)

# datasets["train"] = datasets_train_validation["train"]
# datasets["validation"] = datasets_train_validation["test"]
# datasets["test"] = datasets_train_test["test"]

# datasets

In [ ]:
n_samples_train = len(datasets["train"])
n_samples_validation = len(datasets["validation"])
n_samples_test = len(datasets["test"])
n_samples_total = n_samples_train + n_samples_validation + n_samples_test

print(f"- Training set: {n_samples_train*100/n_samples_total:.2f}%")
print(f"- Validation set: {n_samples_validation*100/n_samples_total:.2f}%")
print(f"- Test set: {n_samples_test*100/n_samples_total:.2f}%")

In [ ]:
# keep only a subsample of the datasets
datasets["train"] = datasets["train"].shuffle().select(range(100000))
datasets["validation"] = datasets["validation"].shuffle().select(range(1000))
datasets["test"] = datasets["test"].shuffle().select(range(1000))

datasets

## Data preprocessing

In [ ]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

In [ ]:
model_checkpoint = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
prefix = "text2text-generation: "

max_input_length = 512
max_target_length = 64

def clean_text(text):
  sentences = nltk.sent_tokenize(text.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
  sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
                                 if len(sent) > 0 and
                                 sent[-1] in string.punctuation]
  text_cleaned = "\n".join(sentences_cleaned_no_titles)
  return text_cleaned

def preprocess_data(examples):
  texts_cleaned = [clean_text(text) for text in examples["text"]]
  inputs = [prefix + text for text in texts_cleaned]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["title"], max_length=max_target_length,
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
datasets_cleaned = datasets.filter(lambda example: (len(example['text']) >= 500) and (len(example['title']) >= 20))
tokenized_datasets = datasets_cleaned.map(preprocess_data, batched=True)
tokenized_datasets

## Fine-tune bart

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
!rm -r {model_dir}

In [ ]:
batch_size = 8
model_name = "bart-base-paraphrasing"
model_dir = f"drive/MyDrive/Models/{model_name}"
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
import numpy as np

metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# Start TensorBoard before training to monitor it in progress
%load_ext tensorboard
%tensorboard --logdir '{model_dir}'/runs

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

## Load the model from GDrive

In [ ]:
model_name = "t5-base-medium-title-generation/checkpoint-2000"
model_dir = f"drive/MyDrive/Models/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

In [ ]:
text = """
We define access to a Streamlit app in a browser tab as a session.
For each browser tab that connects to the Streamlit server, a new session is created.
Streamlit reruns your script from top to bottom every time you interact with your app.
Each reruns takes place in a blank slate: no variables are shared between runs.
Session State is a way to share variables between reruns, for each user session.
In addition to the ability to store and persist state, Streamlit also exposes the
ability to manipulate state using Callbacks. In this guide, we will illustrate the
usage of Session State and Callbacks as we build a stateful Counter app.
For details on the Session State and Callbacks API, please refer to our Session
State API Reference Guide. Also, check out this Session State basics tutorial
video by Streamlit Developer Advocate Dr. Marisa Smith to get started:
"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Session State and Callbacks in Streamlit

In [ ]:
text = """
Many financial institutions started building conversational AI, prior to the Covid19
pandemic, as part of a digital transformation initiative. These initial solutions
were high profile, highly personalized virtual assistants — like the Erica chatbot
from Bank of America. As the pandemic hit, the need changed as contact centers were
under increased pressures. As Cathal McGloin of ServisBOT explains in “how it started,
and how it is going,” financial institutions were looking for ways to automate
solutions to help get back to “normal” levels of customer service. This resulted
in a change from the “future of conversational AI” to a real tactical assistant
that can help in customer service. Haritha Dev of Wells Fargo, saw a similar trend.
Banks were originally looking to conversational AI as part of digital transformation
to keep up with the times. However, with the pandemic, it has been more about
customer retention and customer satisfaction. In addition, new use cases came about
as a result of Covid-19 that accelerated adoption of conversational AI. As Vinita
Kumar of Deloitte points out, banks were dealing with an influx of calls about new
concerns, like questions around the Paycheck Protection Program (PPP) loans. This
resulted in an increase in volume, without enough agents to assist customers, and
tipped the scale to incorporate conversational AI. When choosing initial use cases
to support, financial institutions often start with high volume, low complexity
tasks. For example, password resets, checking account balances, or checking the
status of a transaction, as Vinita points out. From there, the use cases can evolve
as the banks get more mature in developing conversational AI, and as the customers
become more engaged with the solutions. Cathal indicates another good way for banks
to start is looking at use cases that are a pain point, and also do not require a
lot of IT support. Some financial institutions may have a multi-year technology
roadmap, which can make it harder to get a new service started. A simple chatbot
for document collection in an onboarding process can result in high engagement,
and a high return on investment. For example, Cathal has a banking customer that
implemented a chatbot to capture a driver’s license to be used in the verification
process of adding an additional user to an account — it has over 85% engagement
with high satisfaction. An interesting use case Haritha discovered involved
educating customers on financial matters. People feel more comfortable asking a
chatbot what might be considered a “dumb” question, as the chatbot is less judgmental.
Users can be more ambiguous with their questions as well, not knowing the right
words to use, as chatbot can help narrow things down.
"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Conversational AI: The Future of Customer Service

## Upload the model to the Hugging Space Hub

https://huggingface.co/docs/transformers/model_sharing

In [ ]:
!pip install --upgrade jax jaxlib # CPU-only
!pip install flax

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import T5ForConditionalGeneration, TFT5ForConditionalGeneration, FlaxT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained(model_dir)
pt_model = T5ForConditionalGeneration.from_pretrained(model_dir)
tf_model = TFT5ForConditionalGeneration.from_pretrained(model_dir, from_pt=True)
flax_model = FlaxT5ForConditionalGeneration.from_pretrained(model_dir, from_pt=True)

tokenizer.push_to_hub(model_name)
pt_model.push_to_hub(model_name)
tf_model.push_to_hub(model_name)
flax_model.push_to_hub(model_name)

## Load the model from the Hugging Face Hub

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

tokenizer = AutoTokenizer.from_pretrained("fabiochiu/t5-small-medium-title-generation")
model = AutoModelForSeq2SeqLM.from_pretrained("fabiochiu/t5-small-medium-title-generation")

text = """
Many financial institutions started building conversational AI, prior to the Covid19
pandemic, as part of a digital transformation initiative. These initial solutions
were high profile, highly personalized virtual assistants — like the Erica chatbot
from Bank of America. As the pandemic hit, the need changed as contact centers were
under increased pressures. As Cathal McGloin of ServisBOT explains in “how it started,
and how it is going,” financial institutions were looking for ways to automate
solutions to help get back to “normal” levels of customer service. This resulted
in a change from the “future of conversational AI” to a real tactical assistant
that can help in customer service. Haritha Dev of Wells Fargo, saw a similar trend.
Banks were originally looking to conversational AI as part of digital transformation
to keep up with the times. However, with the pandemic, it has been more about
customer retention and customer satisfaction. In addition, new use cases came about
as a result of Covid-19 that accelerated adoption of conversational AI. As Vinita
Kumar of Deloitte points out, banks were dealing with an influx of calls about new
concerns, like questions around the Paycheck Protection Program (PPP) loans. This
resulted in an increase in volume, without enough agents to assist customers, and
tipped the scale to incorporate conversational AI. When choosing initial use cases
to support, financial institutions often start with high volume, low complexity
tasks. For example, password resets, checking account balances, or checking the
status of a transaction, as Vinita points out. From there, the use cases can evolve
as the banks get more mature in developing conversational AI, and as the customers
become more engaged with the solutions. Cathal indicates another good way for banks
to start is looking at use cases that are a pain point, and also do not require a
lot of IT support. Some financial institutions may have a multi-year technology
roadmap, which can make it harder to get a new service started. A simple chatbot
for document collection in an onboarding process can result in high engagement,
and a high return on investment. For example, Cathal has a banking customer that
implemented a chatbot to capture a driver’s license to be used in the verification
process of adding an additional user to an account — it has over 85% engagement
with high satisfaction. An interesting use case Haritha discovered involved
educating customers on financial matters. People feel more comfortable asking a
chatbot what might be considered a “dumb” question, as the chatbot is less judgmental.
Users can be more ambiguous with their questions as well, not knowing the right
words to use, as chatbot can help narrow things down.
"""

inputs = ["summarize: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)
# Conversational AI: The Future of Customer Service

## Evaluate the model on the test set

In [ ]:
import torch

# get test split
test_tokenized_dataset = tokenized_datasets["test"]

# pad texts to the same length
def preprocess_test(examples):
  inputs = [prefix + text for text in examples["text"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,
                           padding="max_length")
  return model_inputs

test_tokenized_dataset = test_tokenized_dataset.map(preprocess_test, batched=True)

# prepare dataloader
test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=32)

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
  predictions = model.generate(**batch)
  all_predictions.append(predictions)

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
all_titles = tokenizer(test_tokenized_dataset["title"], max_length=max_target_length,
                       truncation=True, padding="max_length")["input_ids"]

# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)